In [10]:
import pandas as pd
import numpy as np
import math
from scipy.stats import wilcoxon

In [11]:
def regulate_metrics(values, lenght):
    if len(values) != lenght:

        size_of_batch = len(values) / lenght
        if type(size_of_batch) == float:
            size_of_batch = math.ceil(size_of_batch)
            pad = size_of_batch * lenght
            values = np.pad(values, (pad - len(values)))


        new_values = []
        old_i = 0
        for i in range(size_of_batch, len(values), size_of_batch):
            new_values.append(np.sum(values[old_i:i]))
            old_i = i
        

    return new_values

In [12]:
def load_error_and_rank_metrics(algorithms, dataset, emb, lenght, path="results/", bert="default"):

    labels = ['MSE', 'MAE', 'RMSE', 'PRECISION', 
              'RECALL', 'DIVERSITY', 'NOVELTY', 'NDCG']
    
    alg_name_1 = [""]*len(labels) 
    alg_value_1 = [1.0]*len(labels[:3]) + [0.0]*len(labels[3:])

    alg_name_2 = [""]*len(labels)
    alg_value_2 = [1.0]*len(labels[:3]) + [0.0]*len(labels[3:])

    draw = []
    metrics_matrix = []
    
    for algorithm in algorithms:
            if bert != "default":
                algorithm = algorithm + "_" + bert
        
            error_metrics = pd.read_csv(
                path + algorithm + "_" + dataset + "_" + emb + "_results_error.csv")
            
            rank_metrics = pd.read_csv(
                path + algorithm + "_" + dataset + "_" + emb + "_results_rank.csv")
            

            values = np.concatenate([error_metrics.mean().values, 
                                    rank_metrics.mean().values])
            
            for i, m in enumerate(values[:3]):
                if m < alg_value_1[i]:
                    alg_name_1[i] = algorithm
                    alg_value_1[i] = m

                elif m < alg_value_2[i]:
                    alg_name_2[i] = algorithm
                    alg_value_2[i] = m

            for i, m in enumerate(values[3:]):
                idx = i + 3
                if m > alg_value_1[idx]:
                    alg_name_1[idx] = algorithm
                    alg_value_1[idx] = m

                elif m > alg_value_2[idx]:
                    alg_name_2[idx] = algorithm
                    alg_value_2[idx] = m


            metrics_matrix.append(values)

    # for i, (alg1, alg2) in enumerate(zip(alg_name_1, alg_name_2)):
        
    #     alg1_metrics_error = pd.read_csv(
    #             path + alg1 + "_" + dataset + "_" + emb + "_results_error.csv").values.transpose()
            
    #     alg1_metrics_rank = pd.read_csv(
    #             path + alg1 + "_" + dataset + "_" + emb + "_results_rank.csv").values.transpose()
        
    #     alg2_metrics_error = pd.read_csv(
    #             path + alg2 + "_" + dataset + "_" + emb + "_results_error.csv").values.transpose()
            
    #     alg2_metrics_rank = pd.read_csv(
    #             path + alg2 + "_" + dataset + "_" + emb + "_results_rank.csv").values.transpose()
        
    #     if i < 7:
    #         alg1_metric = alg1_metrics_error[i]
    #         alg2_metric = alg2_metrics_error[i]
    #     else:
    #         alg1_metric = alg1_metrics_rank[0]
    #         alg2_metric = alg2_metrics_rank[0]

    #     if len(alg1_metric) > lenght:
    #         alg1_metric = regulate_metrics(alg1_metric, lenght)
    #     if len(alg2_metric) > lenght:
    #         alg2_metric = regulate_metrics(alg2_metric, lenght)

    #     if not (np.array(alg1_metric) - np.array(alg2_metric)).any():
    #         test = 0.0
    #     else:
    #         test = wilcoxon(alg1_metric, alg2_metric).pvalue
            
    #     if test < 0.05:
    #         draw.append('Draw')
    #     else:
    #         draw.append('Winner')


    df = pd.DataFrame(np.array(metrics_matrix),
                      columns=labels, index=algorithms)
    
    # rank = pd.DataFrame({"Metric": labels,
    #                     "Winner":alg_name_1,
    #                      "2 Place": alg_name_2,
    #                      "Draw": draw})
    rank = []

    return df.round(3), rank
    
    # return  error_metrics.mean(), rank_metrics.mean()

In [13]:
algorithms = ["DeepCoNN", "ConvMF", "TARMF", "NARRE", "DAML"]


df, draw = load_error_and_rank_metrics(algorithms, "Digital_Music_data", "word2vec", 3)
df.transpose()

,DeepCoNN,ConvMF,TARMF,NARRE,DAML
MSE,0.346,0.406,0.353,0.324,0.537
MAE,0.370,0.395,0.387,0.347,0.497
RMSE,0.582,0.630,0.588,0.562,0.651
PRECISION,0.473,0.524,0.478,0.505,0.590
RECALL,0.309,0.260,0.310,0.333,0.503
DIVERSITY,0.045,0.050,0.052,0.041,0.209
NOVELTY,0.008,0.016,0.032,0.032,0.000
NDCG,0.266,0.266,0.266,0.266,0.264


In [14]:
draw

[]

In [15]:
algorithms = ["DeepCoNN", "ConvMF", "TARMF", "NARRE", "DAML"]


df, draw = load_error_and_rank_metrics(algorithms, "Digital_Music_data", "word2vec", 3, bert="ZeroShot")
df.transpose()

,DeepCoNN,ConvMF,TARMF,NARRE,DAML
MSE,0.409,0.409,0.409,0.324,0.323
MAE,0.421,0.416,0.432,0.345,0.346
RMSE,0.632,0.632,0.633,0.562,0.562
PRECISION,0.531,0.522,0.531,0.496,0.508
RECALL,0.254,0.250,0.258,0.340,0.345
DIVERSITY,0.361,0.363,0.357,0.363,0.361
NOVELTY,0.047,0.047,0.050,0.043,0.041
NDCG,0.016,0.024,0.016,0.032,0.024


In [16]:
algorithms = ["DeepCoNN", "ConvMF", "TARMF", "NARRE", "DAML"]


df, draw = load_error_and_rank_metrics(algorithms, "Digital_Music_data", "word2vec", 3, bert="FineTunning")
df.transpose()

,DeepCoNN,ConvMF,TARMF,NARRE,DAML
MSE,0.409,0.409,0.409,0.324,0.323
MAE,0.424,0.416,0.433,0.346,0.347
RMSE,0.632,0.632,0.633,0.562,0.561
PRECISION,0.531,0.524,0.529,0.496,0.507
RECALL,0.255,0.251,0.257,0.341,0.345
DIVERSITY,0.360,0.363,0.357,0.362,0.361
NOVELTY,0.047,0.047,0.050,0.044,0.041
NDCG,0.016,0.016,0.016,0.024,0.032


In [17]:
algorithms = ["DeepCoNN", "ConvMF", "NARRE", "D_ATTN", "MPCN",
            "ALFM", "TRANSNET", "A3NCF", "TARMF", "ANR",
            "CARL", "DAML", "CARP", "CARM", "HRDR"]


df, draw = load_error_and_rank_metrics(algorithms, "Digital_Music_data", "word2vec", 133)
df.transpose()

,DeepCoNN,ConvMF,NARRE,D_ATTN,MPCN,ALFM,TRANSNET,A3NCF,TARMF,ANR,CARL,DAML,CARP,CARM,HRDR
MSE,0.346,0.406,0.324,0.508,0.491,0.493,0.493,0.493,0.353,0.547,0.576,0.537,0.616,0.544,0.320
MAE,0.370,0.395,0.347,0.448,0.470,0.479,0.474,0.472,0.387,0.576,0.472,0.497,0.526,0.545,0.336
RMSE,0.582,0.630,0.562,0.706,0.693,0.694,0.694,0.694,0.588,0.736,0.753,0.651,0.780,0.732,0.559
PRECISION,0.473,0.524,0.505,0.497,0.499,0.799,0.799,0.799,0.478,0.491,0.375,0.590,0.333,0.428,0.498
RECALL,0.309,0.260,0.333,0.240,0.227,0.224,0.224,0.224,0.310,0.246,0.225,0.503,0.240,0.223,0.333
DIVERSITY,0.045,0.050,0.041,0.052,0.053,0.053,0.053,0.053,0.052,0.210,0.055,0.209,0.088,0.117,0.368
NOVELTY,0.008,0.016,0.032,0.040,0.040,0.024,0.000,0.024,0.032,0.089,0.048,0.000,0.032,0.048,0.040
NDCG,0.266,0.266,0.266,0.266,0.266,0.300,0.250,0.300,0.266,0.266,0.266,0.264,0.266,0.266,0.032


In [18]:
order = df["DIVERSITY"].sort_values()
score = [i/16 for i in range(16)]
dm = list(zip(order.index, score))

In [19]:
draw

[]

In [20]:
daml = pd.read_csv("results/DAML_Digital_Music_data_word2vec_results_error.csv")
daml_recall = daml['recall'].values

narre = pd.read_csv("results/NARRE_Digital_Music_data_word2vec_results_error.csv")
narre_recall = narre['recall'].values

In [21]:
wilcoxon(regulate_metrics(daml_recall, 133), narre_recall)

WilcoxonResult(statistic=0.0, pvalue=1.4317759392997278e-23)

In [22]:
pd.read_csv("results/DeepCoNN_Tucso_data_word2vec_results_error.csv").shape

(254, 6)

In [23]:
df, draw = load_error_and_rank_metrics(algorithms, "Musical_Instruments_data", "word2vec", 181)
df.transpose()

,DeepCoNN,ConvMF,NARRE,D_ATTN,MPCN,ALFM,TRANSNET,A3NCF,TARMF,ANR,CARL,DAML,CARP,CARM,HRDR
MSE,0.882,0.916,0.861,1.011,0.982,1.103,0.977,0.977,0.876,1.021,1.042,1.000,1.091,1.037,0.860
MAE,0.684,0.700,0.662,0.749,0.763,0.777,0.747,0.746,0.659,0.731,0.769,0.779,0.841,0.777,0.653
RMSE,0.935,0.951,0.923,1.000,0.986,1.045,0.983,0.983,0.931,1.005,1.016,0.933,1.041,1.014,0.923
PRECISION,0.422,0.493,0.407,0.440,0.442,0.426,0.164,0.164,0.385,0.424,0.387,0.375,0.334,0.361,0.417
RECALL,0.244,0.234,0.255,0.206,0.202,0.196,0.200,0.200,0.244,0.202,0.202,0.303,0.209,0.207,0.251
DIVERSITY,0.077,0.073,0.071,0.078,0.082,0.088,0.074,0.074,0.069,0.083,0.089,0.302,0.228,0.101,0.209
NOVELTY,0.008,0.016,0.023,0.008,0.016,0.016,0.000,0.000,0.008,0.016,0.023,0.125,0.008,0.023,0.069
NDCG,0.257,0.257,0.257,0.257,0.257,0.300,0.257,0.300,0.257,0.257,0.257,0.261,0.257,0.257,0.023


In [24]:
order = df["DIVERSITY"].sort_values()
score = [i/16 for i in range(16)]
mi = list(zip(order.index, score))

In [25]:
algorithms = ["DeepCoNN", "ConvMF", "NARRE", "D_ATTN", "MPCN",
            "ALFM", "TRANSNET", "A3NCF", "TARMF", "ANR",
            "CARL", "DAML", "CARP", "CARM"]

df, draw = load_error_and_rank_metrics(algorithms, "Office_Products_data", "word2vec", 625)
df.transpose()

,DeepCoNN,ConvMF,NARRE,D_ATTN,MPCN,ALFM,TRANSNET,A3NCF,TARMF,ANR,CARL,DAML,CARP,CARM
MSE,0.934,0.982,0.929,1.102,1.067,1.028,1.028,1.028,0.910,1.194,1.049,1.039,1.096,1.180
MAE,0.693,0.710,0.670,0.787,0.814,0.754,0.754,0.755,0.664,0.913,0.772,0.778,0.809,0.882
RMSE,0.962,0.985,0.958,1.045,1.029,1.008,1.008,1.008,0.949,1.090,1.019,0.944,1.043,1.083
PRECISION,0.389,0.477,0.385,0.371,0.432,0.152,0.152,0.152,0.371,0.361,0.451,0.315,0.343,0.316
RECALL,0.246,0.226,0.247,0.205,0.202,0.202,0.202,0.202,0.245,0.202,0.213,0.291,0.215,0.202
DIVERSITY,0.082,0.073,0.069,0.101,0.152,0.073,0.073,0.073,0.067,0.353,0.096,0.297,0.162,0.270
NOVELTY,0.031,0.023,0.023,0.016,0.031,0.016,0.000,0.016,0.016,0.055,0.039,0.000,0.023,0.016
NDCG,0.255,0.255,0.255,0.255,0.255,0.300,0.258,0.300,0.255,0.255,0.255,0.263,0.255,0.255


In [26]:
order = df["DIVERSITY"].sort_values()
score = [i/16 for i in range(16)]
op = list(zip(order.index, score))

In [27]:
df, draw = load_error_and_rank_metrics(algorithms, "Tamp_data", "word2vec", 264)
df.transpose()

,DeepCoNN,ConvMF,NARRE,D_ATTN,MPCN,ALFM,TRANSNET,A3NCF,TARMF,ANR,CARL,DAML,CARP,CARM
MSE,3.148979,2.544178,3.05206,3.298637,2.631232,2.411259,2.284815,1.962195,3.327049,2.5487,2.681738,2.109708,2.650584,3.264653
MAE,1.420774,1.337565,1.398704,1.465548,1.363403,1.303224,1.310318,1.184172,1.443682,1.302085,1.315917,1.265946,1.311928,1.426284
RMSE,1.771219,1.592828,1.743728,1.812914,1.619643,1.550818,1.510014,1.39873,1.82065,1.593671,1.634654,1.42979,1.624971,1.803428
PRECISION,0.193738,0.216085,0.187151,0.202377,0.215635,0.224688,0.219505,0.230433,0.203626,0.201455,0.193358,0.17551,0.203741,0.205845
RECALL,0.201875,0.203537,0.198795,0.207763,0.20372,0.208687,0.205408,0.2,0.204965,0.201602,0.199379,0.224872,0.200011,0.204206
DIVERSITY,0.253153,0.312279,0.202577,0.409605,0.360697,0.255868,0.348541,0.120118,0.280943,0.141318,0.139461,0.475576,0.130328,0.210251
NOVELTY,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NDCG,0.256796,0.256796,0.256796,0.256796,0.256796,0.3,NaN,0.3,0.256796,0.256796,0.256796,NaN,0.256796,0.256796


In [28]:
order = df["DIVERSITY"].sort_values()
score = [i/16 for i in range(16)]
tampa = list(zip(order.index, score))

In [29]:
df, _ = load_error_and_rank_metrics(algorithms, "Philladelphi_data", "word2vec", 610)
df.transpose()

,DeepCoNN,ConvMF,NARRE,D_ATTN,MPCN,ALFM,TRANSNET,A3NCF,TARMF,ANR,CARL,DAML,CARP,CARM
MSE,1.487,1.416,1.412,1.473,1.586,1.372,1.711,1.711,1.479,1.450,1.475,1.545,1.401,1.450
MAE,0.952,0.952,0.923,0.941,0.995,0.926,1.076,1.075,0.932,0.932,0.951,1.010,0.933,0.923
RMSE,1.217,1.188,1.185,1.211,1.256,1.169,1.306,1.306,1.213,1.201,1.212,1.209,1.181,1.201
PRECISION,0.358,0.346,0.365,0.364,0.322,0.400,0.280,0.280,0.377,0.340,0.339,0.286,0.335,0.363
RECALL,0.261,0.242,0.267,0.254,0.225,0.255,0.200,0.200,0.272,0.249,0.247,0.275,0.262,0.268
DIVERSITY,0.085,0.084,0.084,0.097,0.092,0.089,0.098,0.098,0.091,0.137,0.095,0.350,0.083,0.110
NOVELTY,0.008,0.008,0.000,0.008,0.008,0.008,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.008
NDCG,0.256,0.256,0.256,0.256,0.256,0.300,0.256,0.300,0.256,0.256,0.256,0.252,0.256,0.256


In [30]:
order = df["DIVERSITY"].sort_values()
score = [i/16 for i in range(16)]
phila = list(zip(order.index, score))

In [31]:
df, _ = load_error_and_rank_metrics(algorithms, "Tucso_data", "word2vec", 254)
# df = df.fillna(0.006).round(3)
# df['DIVERSITY'] = df['DIVERSITY'] * -1
df.transpose()

,DeepCoNN,ConvMF,NARRE,D_ATTN,MPCN,ALFM,TRANSNET,A3NCF,TARMF,ANR,CARL,DAML,CARP,CARM
MSE,3.335826,2.771728,3.26565,3.235485,2.784732,2.65285,2.405901,2.202155,3.60963,2.940495,2.964225,2.357037,3.069794,3.693929
MAE,1.480675,1.423322,1.4679,1.438755,1.41126,1.38028,1.324038,1.269399,1.527806,1.443678,1.447326,1.355226,1.49342,1.557453
RMSE,1.823936,1.663207,1.804532,1.79606,1.667011,1.627009,1.54949,1.482356,1.897377,1.71279,1.719659,1.514538,1.750051,1.919259
PRECISION,0.185631,0.20055,0.182447,0.197498,0.201432,0.253406,0.134707,0.205995,0.192221,0.191507,0.205147,0.151077,0.18371,0.17531
RECALL,0.201786,0.193582,0.203563,0.201959,0.199651,0.200349,0.203184,0.2,0.202352,0.198353,0.203236,0.184131,0.195204,0.201454
DIVERSITY,0.185481,0.273338,0.180342,0.134518,0.226677,0.167648,0.154606,0.108166,0.257854,0.261247,0.287365,0.475608,0.421708,0.317789
NOVELTY,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NDCG,0.258356,0.258356,0.258356,0.258356,0.258356,0.3,NaN,0.3,0.258356,0.258356,0.258356,0.256842,0.258356,0.258356


In [32]:
order = df["DIVERSITY"].sort_values()
score = [i/16 for i in range(16)]
tuc = list(zip(order.index, score))

# Results Analysis

In [62]:
alg = ["convmf", "deepconn", "transnet", "dattn", "narre", "alfm", "tarmf", "a3ncf", "mpcn", "anr", "carl", "daml", "carp", "hrdr", "carm", "man"]
values = sorted([i/16 for i in range(1,17)], reverse=True)
values

[1.0,
 0.9375,
 0.875,
 0.8125,
 0.75,
 0.6875,
 0.625,
 0.5625,
 0.5,
 0.4375,
 0.375,
 0.3125,
 0.25,
 0.1875,
 0.125,
 0.0625]

In [63]:
div = "0.050 & 0.045 & 0.053 & 0.052 & 0.041 & 0.053 & 0.052 & 0.053 & 0.053 & 0.210 & 0.055 & 0.209 & 0.088 & 0.051 & 0.117 & 0.079"
nov = "0.071 & 0.061 & 0.053 & 0.043 & 0.043 & 0.013 & 0.155 & 0.012 & 0.048 & 0.049 & 0.046 & 0.175 & 0.041 & 0.040 & 0.044 & 0.044"

In [121]:
def metric(alg, values, score, error):
    score = score.split("&")
    if error:
        score = [float((x.strip()) * (-1)) for x in score]
    else:
        score = [float(x.strip()) for x in score]
    score_rank = list(zip(alg, score))
    score_list = sorted(score_rank, key=lambda x: x[1], reverse=True)
    score_list = [x[0] for x in score_list]
    score_rank = {alg:score for alg, score in zip(score_list, score)}

    return score_rank

In [155]:
def calc(mse, rmse, mae, prec, rec, ndcg, div, nov):
    alg = ["convmf", "deepconn", "transnet", "dattn", "narre", "alfm", "tarmf", "a3ncf", "mpcn", "anr", "carl", "daml", "carp", "hrdr", "carm", "man"]
    values = sorted([i/16 for i in range(1,17)], reverse=True)

    mse = mse.split("&")
    mse = [x.strip() for x in mse]
    mse_rank = list(zip(alg, mse))
    mse_list = sorted(mse_rank, key=lambda x: x[1])
    mse_list = [x[0] for x in mse_list]
    mse_rank = {alg:score for alg, score in zip(mse_list, values)}

    rmse = rmse.split("&")
    rmse = [x.strip() for x in rmse]
    rmse_rank = list(zip(alg, rmse))
    rmse_list = sorted(rmse_rank, key=lambda x: x[1])
    rmse_list = [x[0] for x in rmse_list]
    rmse_rank = {alg:score for alg, score in zip(rmse_list, values)}

    mae = mae.split("&")
    mae = [x.strip() for x in mae]
    mae_rank = list(zip(alg, mae))
    mae_list = sorted(mae_rank, key=lambda x: x[1])
    mae_list = [x[0] for x in mae_list]
    mae_rank = {alg:score for alg, score in zip(mae_list, values)}

    prec = prec.split("&")
    prec = [x.strip() for x in prec]
    prec_rank = list(zip(alg, prec))
    prec_list = sorted(prec_rank, key=lambda x: x[1], reverse=True)
    prec_list = [x[0] for x in prec_list]
    prec_rank = {alg:score for alg, score in zip(prec_list, values)}

    rec = rec.split("&")
    rec = [x.strip() for x in rec]
    rec_rank = list(zip(alg, rec))
    rec_list = sorted(rec_rank, key=lambda x: x[1], reverse=True)
    rec_list = [x[0] for x in rec_list]
    rec_rank = {alg:score for alg, score in zip(rec_list, values)}

    ndcg = ndcg.split("&")
    ndcg = [x.strip() for x in ndcg]
    ndcg_rank = list(zip(alg, ndcg))
    ndcg_list = sorted(ndcg_rank, key=lambda x: x[1], reverse=True)
    ndcg_list = [x[0] for x in ndcg_list]
    ndcg_rank = {alg:score for alg, score in zip(ndcg_list, values)}

    div = div.split("&")
    div = [x.strip() for x in div]
    div_rank = list(zip(alg, div))
    div_list = sorted(div_rank, key=lambda x: x[1], reverse=True)
    div_list = [x[0] for x in div_list]
    div_rank = {alg:score for alg, score in zip(div_list, values)}

    nov = nov.split("&")
    nov = [x.strip() for x in nov]
    nov_rank = list(zip(alg, nov))
    nov_list = sorted(nov_rank, key=lambda x: x[1], reverse=True)
    nov_list = [x[0] for x in nov_list]
    nov_rank = {alg:score for alg, score in zip(nov_list, values)}

    error_maut = {}
    for alg, score in mse_rank.items():
        error_maut[alg] = score + rmse_rank[alg] + mae_rank[alg]

    rank_maut = {}
    for alg, score in prec_rank.items():
        rank_maut[alg] = score + rec_rank[alg] + ndcg_rank[alg]

    div_maut = {}
    for alg, score in div_rank.items():
        div_maut[alg] = score + nov_rank[alg]

    return error_maut, rank_maut, div_maut

In [159]:
r1 = calc(
    "0.411 & 0.347 & 0.492 & 0.347 & 0.321 & 0.493 & 0.376 & 0.493 & 0.405 &  0.379 & 0.324 & 0.382 & 0.302 & 0.320 & 0.349 & 0.321",
    "0.452 & 0.384 & 0.474 & 0.347 & 0.345 & 0.479 & 0.461 & 0.472 & 0.421 & 0.373 & 0.336 & 0.403 & 0.316 & 0.336 & 0.371 & 0.334",  
    "0.634 & 0.583 & 0.694 & 0.583 & 0.559 & 0.694 & 0.608 & 0.694 & 0.630  & 0.608 & 0.562 & 0.541 & 0.542 & 0.559 & 0.583 & 0.566",
    "0.525 & 0.463 & 0.799 & 0.481 & 0.497 & 0.741 & 0.460 & 0.744 & 0.525 & 0.489 & 0.502 & 0.685 & 0.511 & 0.498 & 0.451 & 0.530",
    "0.274 & 0.319 & 0.224 & 0.304 & 0.345 & 0.224 & 0.339 & 0.204 & 0.261 & 0.286 & 0.331 & 0.614 & 0.366 & 0.333 & 0.366 & 0.398", 
    "0.000 & 0.000 & 0.000 & 0.024 & 0.016 & 0.016 & 0.032 & 0.024 & 0.040 & 0.024 & 0.016 & 0.000 & 0.024 & 0.032 & 0.032 & 0.033",  
    "0.050 & 0.045 & 0.053 & 0.052 & 0.041 & 0.053 & 0.052 & 0.053 & 0.053 & 0.210 & 0.055 & 0.209 & 0.088 & 0.051 & 0.117 & 0.079", 
    "0.071 & 0.061 & 0.053 & 0.043 & 0.043 & 0.013 & 0.155 & 0.012 & 0.048 & 0.049 & 0.046 & 0.175 & 0.041 & 0.040 & 0.044 & 0.044"
)

r2 = calc(
        "0.927 & 0.884 & 0.977 & 0.889 & 0.862 & 0.841 & 0.882 & 0.977 & 0.922 & 0.903 & 0.854 & 0.925 & 0.884 & 0.860 & 0.885 & 0.894",
        "0.753 & 0.667 & 0.747 & 0.685 & 0.670 & 0.637 & 0.720 & 0.746 & 0.718 & 0.650 & 0.658 & 0.716 & 0.668 & 0.653 & 0.666 & 0.655", 
        "0.959 & 0.935 & 0.983 & 0.939 & 0.924 & 0.912 & 0.935 & 0.984 & 0.955 & 0.945 & 0.919 & 0.887 & 0.935 & 0.923 & 0.936 & 0.989",
        "0.507 & 0.442 & 0.364 & 0.402 & 0.414 & 0.535 & 0.386 & 0.154 & 0.509 & 0.497 & 0.455 & 0.491 & 0.434 & 0.417 & 0.385 & 0.522",
        "0.222 & 0.243 & 0.200 & 0.247 & 0.256 & 0.262 & 0.244 & 0.203 & 0.234 & 0.239 & 0.256 & 0.386 & 0.239 & 0.251 & 0.245 & 0.289",
        "0.008 & 0.023 & 0.000 & 0.023 & 0.023 & 0.016 & 0.016 & 0.000 & 0.023 & 0.023 & 0.023 & 0.000 & 0.023 & 0.023 & 0.031 & 0.023",
        "0.073 & 0.077 & 0.074 & 0.078 & 0.071 & 0.088 & 0.069 & 0.074 & 0.082 & 0.083 & 0.089 & 0.302 & 0.228 & 0.110 & 0.101 & 0.090",
        "0.119 & 0.070 & 0.074 & 0.075 & 0.075 & 0.029 & 0.134 & 0.034 & 0.072 & 0.106 & 0.069 & 0.281 & 0.074 & 0.069 & 0.069 & 0.078"
)

r3 = calc(
    "0.984 & 0.933 & 1.028 & 0.928 & 0.922 & 1.028 & 0.922 & 1.028 & 0.970 & 0.946 & 0.943 & 1.002 & 0.930 & 0.863 & 0.927 & 0.886",
    "0.731 & 0.687 & 0.754 & 0.664 & 0.676 & 0.754 & 0.717 & 0.755 & 0.706 & 0.709 & 0.707 & 0.741 & 0.676 & 0.703 & 0.701 & 0.716",
    "0.986 & 0.961 & 1.008 & 0.958 & 0.955 & 1.008 & 0.956 & 1.008 & 0.979 & 0.967 & 0.966 & 0.921 & 0.959 & 0.928 & 0.958 & 0.945",
    "0.474 & 0.399 & 0.152 & 0.385 & 0.379 & 0.152 & 0.371 & 0.152 & 0.458 & 0.466 & 0.490 & 0.461 & 0.373 & 0.391 & 0.337 & 0.410",
    "0.226 & 0.244 & 0.202 & 0.243 & 0.248 & 0.202 & 0.248 & 0.202 & 0.231 &  0.236 & 0.246 & 0.350 & 0.240 & 0.301 & 0.255 & 0.298",
    "0.023 & 0.023 & 0.010 & 0.016 & 0.023 & 0.016 & 0.016 & 0.016 & 0.008 & 0.008 & 0.016 & 0.000 & 0.016 & 0.023 & 0.039 & 0.008",
    "0.073 & 0.082 & 0.073 & 0.101 & 0.069 & 0.073 & 0.067 & 0.073 & 0.152 & 0.353 & 0.096 & 0.297 & 0.162 & 0.170 & 0.270 & 0.169",
    "0.074 & 0.075 & 0.073 & 0.071 & 0.070 & 0.023 & 0.129 & 0.023 & 0.071 & 0.075 & 0.076 & 0.286 & 0.071 & 0.069 & 0.106 & 0.061"
)

mean_err = {}
for alg, value in r1[0].items():
    mean_err[alg] = value + r2[0][alg] + r3[0][alg]

mean_rank = {}
for alg, value in r1[1].items():
    mean_rank[alg] = value + r2[1][alg] + r3[1][alg]

mean_div = {}
for alg, value in r1[2].items():
    mean_div[alg] = value + r2[2][alg] + r3[2][alg]

mean_err

{'carp': 6.75,
 'hrdr': 7.75,
 'narre': 7.1875,
 'man': 6.0,
 'carl': 6.4375,
 'deepconn': 5.6875,
 'dattn': 5.625,
 'carm': 5.5625,
 'tarmf': 4.75,
 'anr': 4.375,
 'daml': 4.9375,
 'mpcn': 3.25,
 'convmf': 2.1875,
 'transnet': 1.5,
 'alfm': 3.625,
 'a3ncf': 0.875}

In [160]:
mean_rank

{'transnet': 2.375,
 'a3ncf': 2.5625,
 'alfm': 4.5,
 'daml': 5.5,
 'man': 6.625,
 'convmf': 4.625,
 'mpcn': 5.0625,
 'carp': 4.5,
 'carl': 5.4375,
 'hrdr': 5.75,
 'narre': 5.5625,
 'anr': 4.5,
 'dattn': 4.875,
 'deepconn': 4.875,
 'tarmf': 4.3125,
 'carm': 5.4375}

In [161]:
mean_div

{'anr': 4.75,
 'daml': 5.875,
 'carm': 4.125,
 'carp': 3.5625,
 'man': 3.625,
 'carl': 3.5625,
 'transnet': 3.125,
 'alfm': 1.75,
 'a3ncf': 1.1875,
 'mpcn': 3.0625,
 'dattn': 2.9375,
 'tarmf': 3.25,
 'hrdr': 2.625,
 'convmf': 3.125,
 'deepconn': 2.875,
 'narre': 1.5625}

In [158]:
r1[0]

{'carp': 2.9375,
 'hrdr': 2.5625,
 'narre': 2.5,
 'man': 2.4375,
 'carl': 2.375,
 'deepconn': 1.8125,
 'dattn': 1.875,
 'carm': 1.6875,
 'tarmf': 1.1875,
 'anr': 1.375,
 'daml': 1.8125,
 'mpcn': 1.0,
 'convmf': 0.8125,
 'transnet': 0.5,
 'alfm': 0.3125,
 'a3ncf': 0.3125}

In [162]:
r1 = calc(
    "1.840 & 1.849 & 2.156 & 1.847 & 1.786 & 1.700 & 1.856 & 2.202 & 1.964 & 1.793 & 1.844 & 1.925 & 1.772 & 1.698 & 1.858 & 1.702",
	"1.144 & 1.078 & 1.239 & 1.073 & 1.060 & 1.057 & 1.081 & 1.239 & 1.162 & 1.091 & 1.107 & 1.168 & 1.078 & 1.057 & 1.089 & 1.001",
	"1.355 & 1.357 & 1.467 & 1.356 & 1.333 & 1.301 & 1.359 & 1.482 & 1.399 & 1.337 & 1.355 & 1.359 & 1.328 & 1.302 & 1.360 & 1.299",
	"0.321 & 0.349 & 0.134 & 0.354 & 0.365 & 0.379 & 0.380 & 0.206 & 0.312 & 0.355 & 0.367 & 0.235 & 0.337 & 0.337 & 0.362 & 0.346",
	"0.237 & 0.272 & 0.208 & 0.265 & 0.275 & 0.252 & 0.286 & 0.203 & 0.231 &  0.259 & 0.271 & 0.237 & 0.265 & 0.234 & 0.280 & 0.190",
	"0.000 & 0.000 & 0.006 & 0.010 & 0.038 & 0.009 & 0.000 & 0.008 & 0.008 & 0.000 & 0.015 & 0.000 & 0.015 & 0.010 & 0.008 & 0.010",
	"0.312 & 0.253 & 0.348 & 0.409 & 0.202 & 0.255 & 0.280 & 0.120 & 0.360 & 0.141 & 0.139 & 0.475 & 0.130 & 0.219 & 0.210 & 0.122",
	"0.139 & 0.102 & 0.111 & 0.120 & 0.100 & 0.006 & 0.134 & 0.008 & 0.104 & 0.095 & 0.155 & 0.392 & 0.097 & 0.110 & 0.133 & 0.095"
)

r2 = calc(
        "1.647 & 1.696 & 1.756 & 1.715 & 1.631 & 1.533 & 1.702 & 1.962 & 1.791 & 1.657 & 1.720 & 1.737 & 1.600 & 1.695 & 1.710 & 1.629", 
        "1.056 & 1.008 & 1.082 & 1.038 & 0.988 & 0.997 & 1.007 & 1.184 & 1.081 & 1.002 & 1.041 & 1.088 & 0.977 & 1.003 & 1.021 & 1.001",
        "1.281 & 1.299 & 1.323 & 1.307 & 1.274 & 1.236 & 1.302 & 1.399 & 1.336 & 1.284 & 1.309 & 1.285 & 1.262 & 1.299 & 1.305 & 1.276",
        "0.347 & 0.359 & 0.288 & 0.379 & 0.372 & 0.313 & 0.384 & 0.230 & 0.322 & 0.318 & 0.318 & 0.266 & 0.330 & 0.310 & 0.380 & 0.295",
        "0.239 & 0.276 & 0.222 & 0.280 & 0.276 & 0.255 & 0.284 & 0.200 & 0.235 & 0.251 & 0.253 & 0.261 & 0.262 & 0.274 & 0.272 & 0.304",
        "0.019 & 0.016 & 0.000 & 0.016 & 0.038 & 0.010 & 0.016 & 0.018 & 0.010 & 0.016 & 0.016 & 0.000 & 0.016 & 0.013 & 0.008 & 0.011",
        "0.045 & 0.120 & 0.120 & 0.036 & 0.133 & 0.103 & 0.085 & 0.067 & 0.082 & 0.192 & 0.116 & 0.112 & 0.184 & 0.109 & 0.087 & 0.111",
        "0.141 & 0.094 & 0.095 & 0.156 & 0.093 & 0.021 & 0.103 & 0.020 & 0.107 & 0.152 & 0.096 & 0.384 & 0.140 & 0.090 & 0.105 & 0.095"
)

r3 = calc(
        "1.416 & 1.487 & 1.611 & 1.473 & 1.412 & 1.372 & 1.479 & 1.611 & 1.586 & 1.450 & 1.475 & 1.545 & 1.401 & 1.451 & 1.450 & 1.420",
        "0.952 & 0.952 & 1.076 & 0.941 & 0.923 & 0.926 & 0.932 & 1.075 & 0.995 & 0.932 & 0.951 & 1.010 & 0.933 & 0.952 & 0.923 & 0.976",
        "1.188 & 1.217 & 1.206 & 1.211 & 1.185 & 1.169 & 1.213 & 1.206 & 1.256 & 1.201 & 1.212 & 1.209 & 1.181 & 1.210 & 1.201 & 1.180",
        "0.346 & 0.358 & 0.290 & 0.364 & 0.365 & 0.400 & 0.377 & 0.301 & 0.322 & 0.340 & 0.339 & 0.286 & 0.335 & 0.310 & 0.363 & 0.358",
        "0.242 & 0.261 & 0.200 & 0.254 & 0.267 & 0.255 & 0.272 & 0.200 & 0.225 & 0.249 & 0.247 & 0.275 & 0.262 & 0.262 & 0.268 & 0.260",
        "0.000 & 0.001 & 0.013 & 0.019 & 0.000 & 0.012 & 0.013 & 0.002 & 0.012 & 0.002 & 0.014 & 0.019 & 0.018 & 0.027 & 0.019 & 0.026",
        "0.084 & 0.085 & 0.098 & 0.097 & 0.084 & 0.091 & 0.092 & 0.137 & 0.095 & 0.350 & 0.083 & 0.117 & 0.222 & 0.272 & 0.110 & 0.137",
        "0.084 & 0.085 & 0.098 & 0.097 & 0.084 & 0.049 & 0.091 & 0.018 & 0.092 & 0.137 & 0.095 & 0.350 & 0.083 & 0.097 & 0.110 & 0.091"
)

mean_err = {}
for alg, value in r1[0].items():
    mean_err[alg] = value + r2[0][alg] + r3[0][alg]

mean_rank = {}
for alg, value in r1[1].items():
    mean_rank[alg] = value + r2[1][alg] + r3[1][alg]

mean_div = {}
for alg, value in r1[2].items():
    mean_div[alg] = value + r2[2][alg] + r3[2][alg]

mean_err

{'hrdr': 5.875,
 'alfm': 8.5625,
 'man': 7.375,
 'carp': 7.625,
 'narre': 7.625,
 'anr': 6.0625,
 'convmf': 5.4375,
 'carl': 3.6875,
 'dattn': 4.3125,
 'deepconn': 4.125,
 'tarmf': 4.25,
 'carm': 4.5625,
 'daml': 2.625,
 'mpcn': 1.625,
 'transnet': 1.6875,
 'a3ncf': 1.0625}

In [163]:
mean_rank

{'tarmf': 7.1875,
 'alfm': 5.125,
 'carl': 5.3125,
 'narre': 7.0,
 'carm': 6.25,
 'anr': 3.9375,
 'dattn': 6.75,
 'deepconn': 5.5,
 'man': 5.0,
 'carp': 5.4375,
 'hrdr': 4.8125,
 'convmf': 3.875,
 'mpcn': 2.875,
 'daml': 3.1875,
 'a3ncf': 2.375,
 'transnet': 1.875}

In [164]:
mean_div

{'daml': 5.3125,
 'dattn': 3.875,
 'mpcn': 3.3125,
 'transnet': 4.0,
 'convmf': 3.0625,
 'tarmf': 3.25,
 'alfm': 1.6875,
 'deepconn': 2.75,
 'hrdr': 3.375,
 'carm': 3.6875,
 'narre': 2.25,
 'anr': 4.375,
 'carl': 3.0625,
 'carp': 3.25,
 'man': 2.4375,
 'a3ncf': 1.3125}

In [136]:
def mean(b1, b2, b3):
    err, rank, div = calc(b1[0], b1[1], b1[2], b1[3], b1[4], b1[5], b1[6], b1[7]), calc(b2[0], b2[1], b2[2], b2[3], b2[4], b2[5], b2[6], b2[7]), calc(b3[0], b3[1], b3[2], b3[3], b3[4], b3[5], b3[6], b3[7])


    return maut

In [143]:
amazon = mean(
    [
        "0.050 & 0.045 & 0.053 & 0.052 & 0.041 & 0.053 & 0.052 & 0.053 & 0.053 & 0.210 & 0.055 & 0.209 & 0.088 & 0.051 & 0.117 & 0.079",
        "0.071 & 0.061 & 0.053 & 0.043 & 0.043 & 0.013 & 0.155 & 0.012 & 0.048 & 0.049 & 0.046 & 0.175 & 0.041 & 0.040 & 0.044 & 0.044"
    ],
    [
        "0.073 & 0.077 & 0.074 & 0.078 & 0.071 & 0.088 & 0.069 & 0.074 & 0.082 & 0.083 & 0.089 & 0.302 & 0.228 & 0.110 & 0.101 & 0.090",
        "0.119 & 0.070 & 0.074 & 0.075 & 0.075 & 0.029 & 0.134 & 0.034 & 0.072 & 0.106 & 0.069 & 0.281 & 0.074 & 0.069 & 0.069 & 0.078"
    ],
    [
        "0.073 & 0.082 & 0.073 & 0.101 & 0.069 & 0.073 & 0.067 & 0.073 & 0.152 & 0.353 & 0.096 & 0.297 & 0.162 & 0.170 & 0.270 & 0.169",
        "0.074 & 0.075 & 0.073 & 0.071 & 0.070 & 0.023 & 0.129 & 0.023 & 0.071 & 0.075 & 0.076 & 0.286 & 0.071 & 0.069 & 0.106 & 0.061"
    ]
)

TypeError: calc() missing 6 required positional arguments: 'mae', 'prec', 'rec', 'ndcg', 'div', and 'nov'

In [138]:
amazon

{'convmf': 3.125,
 'deepconn': 3.5,
 'transnet': 3.0,
 'dattn': 3.625,
 'narre': 2.0,
 'alfm': 3.125,
 'tarmf': 1.5,
 'a3ncf': 2.0,
 'mpcn': 3.25,
 'anr': 4.0,
 'carl': 3.125,
 'daml': 4.5,
 'carp': 3.75,
 'hrdr': 3.75,
 'carm': 3.625,
 'man': 3.125}

In [139]:
yelp = mean(
    [
        "0.312 & 0.253 & 0.348 & 0.409 & 0.202 & 0.255 & 0.280 & 0.120 & 0.360 & 0.141 & 0.139 & 0.475 & 0.130 & 0.219 & 0.210 & 0.122",
	    "0.139 & 0.102 & 0.111 & 0.120 & 0.100 & 0.006 & 0.134 & 0.008 & 0.104 & 0.095 & 0.155 & 0.392 & 0.097 & 0.110 & 0.133 & 0.095"
    ],
    [
        "0.045 & 0.120 & 0.120 & 0.036 & 0.133 & 0.103 & 0.085 & 0.067 & 0.082 & 0.192 & 0.116 & 0.112 & 0.184 & 0.109 & 0.087 & 0.111",
        "0.141 & 0.094 & 0.095 & 0.156 & 0.093 & 0.021 & 0.103 & 0.020 & 0.107 & 0.152 & 0.096 & 0.384 & 0.140 & 0.090 & 0.105 & 0.095"
    ],
    [
        "0.084 & 0.085 & 0.098 & 0.097 & 0.084 & 0.091 & 0.092 & 0.137 & 0.095 & 0.350 & 0.083 & 0.117 & 0.222 & 0.272 & 0.110 & 0.137",
        "0.084 & 0.085 & 0.098 & 0.097 & 0.084 & 0.049 & 0.091 & 0.018 & 0.092 & 0.137 & 0.095 & 0.350 & 0.083 & 0.097 & 0.110 & 0.091"
    ]
)

In [140]:
yelp

{'convmf': 2.625,
 'deepconn': 4.0,
 'transnet': 4.375,
 'dattn': 2.75,
 'narre': 3.5,
 'alfm': 2.875,
 'tarmf': 2.625,
 'a3ncf': 3.125,
 'mpcn': 2.375,
 'anr': 4.875,
 'carl': 2.25,
 'daml': 3.25,
 'carp': 4.125,
 'hrdr': 3.25,
 'carm': 2.25,
 'man': 2.75}

In [141]:
overall = {}

for alg, value in amazon.items():
    overall[alg] = value + yelp[alg]

overall

{'convmf': 5.75,
 'deepconn': 7.5,
 'transnet': 7.375,
 'dattn': 6.375,
 'narre': 5.5,
 'alfm': 6.0,
 'tarmf': 4.125,
 'a3ncf': 5.125,
 'mpcn': 5.625,
 'anr': 8.875,
 'carl': 5.375,
 'daml': 7.75,
 'carp': 7.875,
 'hrdr': 7.0,
 'carm': 5.875,
 'man': 5.875}